# Network One: classifier for music vs. non-music

This notebook contains the first part of the assigment for ECS7013P, Deep Learning for Audio and Music, QMUL.  
It consists of an **implementation of a neural network for binary classification between music and non-music from audio samples**.  
Code and ideas were based, built upon and inspired by:  
- Lecture 3 from Deep Learning for Audio and Music (ECS7013P) on pre-trained features, QMUL, by Dan Stowell.  
- https://github.com/slychief/ismir2018_tutorial 
- https://github.com/keunwoochoi/dl4mir  
- https://github.com/tensorflow/models/tree/master/research/audioset/vggish

Also, this implementation uses pre-trained features, exploring torchvggish's package available at:  
- https://github.com/harritaylor/torchvggish  

Training and testing were conducted using GTZAN and URBANSOUND8K datasets available at:  
- [GTZAN] http://marsyas.info/downloads/datasets.html  
- [URBANSOUND8K] https://urbansounddataset.weebly.com/urbansound8k.html  

As per this coursework's requirements, a test case audio file is needed to evaluate the network's performance. A track from *Conduct*, by October Horse, a progressive metal band from Porto, Portugal was chosen:  
- https://octoberhorse.bandcamp.com/track/waving

In [49]:
import numpy as np
import os

import librosa
import pickle

import torchvggish
from torchvggish import vggish, vggish_input

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

import itertools
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline

import IPython.display as ipd

## Implementation

In [50]:
PATH = os.getcwd()

In [51]:
# Paths to where the datasets (GTZAN and URBANSOUND8K) are stored on local filesystem
path_to_gtzan  = "/import/c4dm-datasets/gtzan/"
path_to_urban = "/import/c4dm-datasets/UrbanSound8K/audio/"

In [52]:
def create_dictionaries(PATH1, PATH2, return_all=True):
    '''
    Function to create dictionaries from datasets.
    Arguments:
        PATH1: path to urban sound 8k dataset
        PATH2: path to gtzan dataset
        return_all: if true returns 3 dictionaries, one for each dataset and one combined
    Return:
        Dictionaries with file path and label (non-music/music)
    '''
    
    path_to_urban = PATH1
    path_to_gtzan = PATH2
    
    # Limit UrbanSound dataset to balance both datasets
    maxfilestoload  = 120  
    
    # UrbanSound Dataset
    # create dictionary with file name as key 
    # and the corresponding label (0, no music)
    urbanfiles_Dictionary ={}

    for entry in os.scandir(path_to_urban):
            if os.path.isdir(entry): 
                counter = 0
                for wavFile in os.scandir(entry.path):
                    counter = counter + 1
                    if (counter <= maxfilestoload): 
                        filename = wavFile.name
                        filename = filename[:-4]
                        # Label it as zero, for non-music
                        urbanfiles_Dictionary[entry.name+"/"+filename] = 0
                    else:
                        break
                    
    ## GTZAN dataset
    # create dictionary with file name as key 
    # and the corresponding label (1, music)
    gtzanfiles_Dictionary ={}
    for entry in os.scandir(path_to_gtzan):
            if os.path.isdir(entry): 
                for wavFile in os.scandir(entry.path):
                    filename = wavFile.name
                    filename = filename[:-3] 
                    # Label it as one, for music
                    gtzanfiles_Dictionary[entry.name+"/"+filename] = 1
    
    # Combine label dictionaries into one
    files_Dictionary = {}
    files_Dictionary.update(urbanfiles_Dictionary)
    files_Dictionary.update(gtzanfiles_Dictionary)
    
    if return_all:
        return files_Dictionary, urbanfiles_Dictionary, gtzanfiles_Dictionary
    else:
        return files_Dictionary


In [53]:
# Create dictionaries
files_Dictionary, urbanfiles_Dictionary, gtzanfiles_Dictionary = create_dictionaries(path_to_urban, path_to_gtzan)

In [ ]:
# Create vggish mel spectograms
# Details about procedure and dimensions can be found at:
#    https://github.com/tensorflow/models/tree/master/research/audioset/vggish
urban_melspecs_vgg = {key:vggish_input.waveform_to_examples(librosa.load("%s/%s.wav" % (path_to_urban, key))[0], librosa.load("%s/%s.wav" % (path_to_urban, key))[1])
for key, label in urbanfiles_Dictionary.items() }
gtzan_melspecs_vgg = {key:vggish_input.waveform_to_examples(librosa.load("%s/%s.au" % (path_to_gtzan, key), duration=4, offset=10)[0], librosa.load("%s/%s.au" % (path_to_gtzan, key), duration=4, offset=10)[1])
for key, label in gtzanfiles_Dictionary.items() }

# Merge mel spectograms into one single dictionary
melspecs = {}
melspecs.update(urban_melspecs_vgg)
melspecs.update(gtzan_melspecs_vgg)

# Save mel spectrograms with pickle 
melspecs_pickle = open(PATH + '/' + 'melspecs.pickle',"wb")
pickle.dump(melspecs, melspecs_pickle)
melspecs_pickle.close()

In [54]:
def load_spectrograms(path):
    '''
    Function to load spectograms from pickle.
    '''
    with open(path+ '/' + 'melspecs.pickle', 'rb') as handle:
        melspecs = pickle.load(handle)
    return melspecs

In [55]:
# Loading mel spectograms
melspecs = load_spectrograms(PATH)

In [56]:
# In both melspecs and label dictionaries, delete entries withshort audio (present in UrbanSound8K)
for itemid in list(melspecs.keys()):
    if melspecs[itemid].shape[0] == 0:
            del melspecs[itemid]
            del urbanfiles_Dictionary[itemid]
            del files_Dictionary[itemid]
            
# To make melspecs with 1000 examples from each dataset            
for itemid in list(urbanfiles_Dictionary.keys()):
    if len(melspecs)>2000:
        del melspecs[itemid]
        del urbanfiles_Dictionary[itemid]
        del files_Dictionary[itemid]

In [57]:
# Shuffle dictionary with file names
keys = list(files_Dictionary.keys())
np.random.shuffle(keys)

# Shuffle melspecs with the new keys
melfeatures_Dict = {}
for key in keys:
    melfeatures_Dict.update({key:melspecs[key]})
    
del melspecs
melspecs = melfeatures_Dict

In [15]:
# UNCOMMENT CELL TO SEE/HEAR ONE EXAMPLE FROM EACH DATASET

# urban_key = 'fold8/156358-5-0-4'   # Engine idling
# gtzan_key = 'metal/metal.00043'    # Short snippet of the amazing Ronnie James Dio

# urban_example_4D = melspecs[urban_key].detach().numpy()
# # Use numpy "concatenate" to join the 1-second chunks back together.
# urban_example_2D = np.concatenate(np.concatenate(urban_example_4D)).T

# gtzan_example_4D = melspecs[gtzan_key].detach().numpy()
# gtzan_example_2D = np.concatenate(np.concatenate(gtzan_example_4D)).T

# # Plotting side-by-side to visually inspect mel spectograms
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))
# axes[0].imshow(urban_example_2D, aspect='auto', origin='lower')
# axes[0].set_title('UrbanSound8k: ' + urban_key)
# axes[0].set_ylabel('Mel bins')
# axes[0].set_xlabel('Time frames')
# axes[1].imshow(gtzan_example_2D, aspect='auto', origin='lower')
# axes[1].set_title('GTZAN: ' + gtzan_key)
# axes[1].set_ylabel('Mel bins')
# axes[1].set_xlabel('Time frames')
# fig.tight_layout()

# # Aurally inspect the files
# print('UrbanSound8k: ' + urban_key)
# ipd.display(ipd.Audio(path_to_urban + urban_key + ".wav"))

# print('GTZAN: ' + gtzan_key)
# ipd.display(ipd.Audio(librosa.load("%s%s.au" % (path_to_gtzan, gtzan_key), duration=4, offset=10)[0], 
#           rate=librosa.load("%s%s.au" % (path_to_gtzan, gtzan_key), duration=4, offset=10)[1]))

In [58]:
# After shuffling, split dataset into (70%) training and (30%) test 
melspecs_test = {}
melspecs_train = {}
split = 0.7
for i, key in enumerate(melspecs.keys()):
    if (i >= (split*len(melspecs))):
        melspecs_test[key]  = melspecs[key]
    else:
        melspecs_train[key] = melspecs[key]      

In [60]:
# Create embeddings, passing data through the VGGish network
# Details about procedure and dimensions can be found at:
#    https://github.com/tensorflow/models/tree/master/research/audioset/vggish
embedding_model = vggish()
embeddings = {key:embedding_model.forward(item).detach().numpy() for key,item in sorted(melspecs.items())}

In [61]:
# PCA for training set
binarylabels_serial_train = []
Xforpca_vgg_train = []
keys = []
for akey in melspecs_train.keys():
        if akey in embeddings:
                if embeddings[akey].shape != (4,128):
                        #print("  warning: skipping item %s because embedding is unexpected shape" % akey)
                        continue # goes to the beginning of the loop
                binarylabels_serial_train.extend([files_Dictionary[akey]] * embeddings[akey].shape[0])
                #print(embeddings[akey].shape[1])
                Xforpca_vgg_train.append(embeddings[akey])
                #print(Xforpca_vgg[0].shape)
                keys.append(akey)

# len(Xforpca_vgg) 2237
# four_sec_chunks = Xforpca_vgg
# this data will be used for training the last layer of the vggish network
Xforpca_vgg_train = np.array(Xforpca_vgg_train).reshape((-1, 128))
# print(Xforpca_vgg.shape) # (4*2237), 128

pca = PCA(n_components=2, whiten=True)
pcadata_vgg_train = pca.fit_transform(Xforpca_vgg_train) # shape (nsamples, nfeatures)

assert len(binarylabels_serial_train) == Xforpca_vgg_train.shape[0], "len(binarylabels_serial) != Xforpca.shape[0]. %i != %i" % (len(binarylabels_serial), Xforpca.shape[0])


In [62]:
# PCA for test set

binarylabels_serial_test = []
Xforpca_vgg_test = []
keys = []
for akey in melspecs_test.keys():
        if akey in embeddings:
                if embeddings[akey].shape != (4,128):
#                         print("  warning: skipping item %s because embedding is unexpected shape" % akey)
                        continue # goes to the beginning of the loop
                binarylabels_serial_test.extend([files_Dictionary[akey]] * embeddings[akey].shape[0])
                #print(embeddings[akey].shape[1])
                Xforpca_vgg_test.append(embeddings[akey])
                #print(Xforpca_vgg[0].shape)
                keys.append(akey)

# this data will be used for training the last layer of the vggish network
Xforpca_vgg_test = np.array(Xforpca_vgg_test).reshape((-1, 128))

pca = PCA(n_components=2, whiten=True)
pcadata_vgg_test = pca.fit_transform(Xforpca_vgg_test) # shape (nsamples, nfeatures)

assert len(binarylabels_serial_test) == Xforpca_vgg_test.shape[0], "len(binarylabels_serial) != Xforpca.shape[0]. %i != %i" % (len(binarylabels_serial), Xforpca.shape[0])


In [65]:
# Function to plot data
def scatterplot(xs, ys, title, datalabels):
    plt.figure(frameon=False, figsize=(5, 5))
    plt.scatter(xs, ys, alpha=0.2,
                c=[{0: 'b', 1: 'g'}[albl] for albl in datalabels],
               )
    plt.title(title)
    plt.tight_layout()

In [70]:
# # UNCOMMENT BELOW FOR PLOTS
# # Plotting PCA Analysis for both test and train
# scatterplot(pcadata_vgg_train[:,0], pcadata_vgg_train[:,1], "2D PCA of VGG train embeddings", binarylabels_serial_train)
# scatterplot(pcadata_vgg_test[:,0], pcadata_vgg_test[:,1], "2D PCA of VGG test embeddings", binarylabels_serial_test)

In [68]:
# Train: Logistic regression applied to the VGG embeddings
lgr = LogisticRegression(solver='liblinear')
lgr.fit(Xforpca_vgg_train, binarylabels_serial_train)
lgrdata_vgg = lgr.predict_proba(Xforpca_vgg_train) # shape (nsamples, nfeatures)
lgrscore_vgg = lgr.score(Xforpca_vgg_train, binarylabels_serial_train) * 100

In [71]:
# UNCOMMENT BELOW FOR PLOTS
# scatterplot(lgrdata_vgg[:,0], range(len(lgrdata_vgg)), "Training accuracy: logistic regression of VGG embeddings (%.1f %%)" % lgrscore_vgg, binarylabels_serial_train)

In [ ]:
# Testing phase
lgrdata_vgg = lgr.predict_proba(Xforpca_vgg_test) # shape (nsamples, nfeatures)
lgrscore_vgg = lgr.score(Xforpca_vgg_test, binarylabels_serial_test) * 100

In [ ]:
# UNCOMMENT BELOW FOR PLOTS
scatterplot(lgrdata_vgg[:,0], range(len(lgrdata_vgg)), "Test Accuracy: Logistic regression of VGG embeddings (%.1f %%)" % lgrscore_vgg, binarylabels_serial_test)

In [31]:
# Extract classification probabilities for each example
examples_prob = []
for i in range(0, len(lgrdata_vgg), 4):
    examples_prob.append(lgrdata_vgg[i:i+4,:])

In [32]:
# Get classifications based on probabilites, for each example
classifications = []
# Keys that were classified as music
music_keys = []
# Keys that were classified as non-music
non_music_keys = []

for i, prob in enumerate(examples_prob):
    # first column -> probability of non music 
    no_music_prob = sum(prob[:,0])/len(prob)
    # second column -> probability of music
    music_prob = sum(prob[:,1])/len(prob)

    if (no_music_prob >= music_prob):
        output_class = 0
        non_music_keys.append(keys[i])
    else:
        output_class = 1
        music_keys.append(keys[i])
    #print(output_class)
    classifications.append(output_class)


In [114]:
# Save music_keys into pickle
music_keys_pickle = open("music_keys.pickle", "wb")
pickle.dump(music_keys, music_keys_pickle)
music_keys_pickle.close()

## Performance evaluation of networkOne on test audio file

In [72]:
# Audio file test case - Waving by October Horse 
audiotest_path = '/homes/pps30/venvs/venv_dl4am_a1/dl4am/assignment/Waving_OH.wav'
y, sr = librosa.load(audiotest_path, mono=True)

ipd.Audio(y, rate=sr)

In [25]:
# Create mel spectograms in vggish format
audiotest_melspecs_vgg = vggish_input.waveform_to_examples(librosa.load(audiotest_path, mono=True)[0], librosa.load(audiotest_path, mono=True)[1])

In [27]:
# Generate embeddings from data
embedding_audiotest = embedding_model.forward(audiotest_melspecs_vgg).detach().numpy()

(228, 128)

In [40]:
# Use trained model to check if it is music or non-music
Xforpca_vgg_audiotest = embedding_audiotest.reshape((-1, 128))
pca = PCA(n_components=2, whiten=True)
audiotest_preds = lgr.predict_proba(Xforpca_vgg_audiotest) # shape (nsamples, nfeatures)

In [47]:
# Count how many seconds are considered music agaisnt how many are not
# Calculate average probabilities for music and non-music

counter_nomusic = 0
counter_yesmusic = 0
probs_nomusic = 0
probs_yesmusic = 0 

for i, prob in enumerate(audiotest_preds):
    probs_nomusic += prob[0]
    probs_yesmusic += prob[1]
    
    if prob[0] > prob[1]:
        counter_nomusic += 1
    else:
        counter_yesmusic += 1        
        
avgProb_nomusic = probs_nomusic / len(audiotest_preds) * 100
avgProb_yesmusic = probs_yesmusic / len(audiotest_preds) * 100

print("Number of seconds as music: ", counter_yesmusic)
print("Number of seconds as non-music: ", counter_nomusic)
print("Probability of being music: ", avgProb_yesmusic)
print("Probability of being non-music: ", avgProb_nomusic)
      

Number of seconds as music:  169
Number of seconds as non-music:  59
Probability of being music:  72.3991263201452
Probability of being non-music:  27.60087367985476
